<a href="https://colab.research.google.com/github/ronaldexim/dw_matrix_road_sign/blob/master/matrix3_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install hyperopt

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import os, datetime
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign/data'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign/data


In [0]:
names = pd.read_csv('signnames.csv')
labels_dict = names.to_dict()['b']

In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [5]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
y_train.shape, y_test.shape

((34799, 43), (4410, 43))

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
params_fit ={
    'epochs': 25
}

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
        model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')

        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

        model.fit(
            X_train, 
            y_train,
            batch_size = params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose = params_fit.get('verbose', 1),
            validation_data = params_fit.get('validation_data', (X_train, y_train)),
            callbacks = [tensorboard_callback]
            )
        return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

    y_test_norm = np.argmax(y_test, axis=1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(),

        Dense(1024, activation='relu'),
        Dropout(0.3),   
        Dense(num_classes, activation='softmax'),
    ])
    model.summary()
    return model

def get_cnn_v6(input_shape, num_classes):
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(),

        Dense(1024, activation='relu'),
        Dropout(0.3),

        Dense(1024, activation='relu'),
        Dropout(0.3),

        Dense(num_classes, activation='softmax'),
    ])
    model.summary()
    return model

In [12]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit)
predict(model_trained, X_test, y_test)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)         

0.9741496598639455

In [14]:
model = get_cnn_v6(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit)
predict(model_trained, X_test, y_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 64)         

0.9816326530612245

In [19]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 64)         

In [0]:
def get_model(params):
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),

        Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),

        Flatten(),

        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_one']),

        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_two']),

        Dense(num_classes, activation='softmax'),
    ])
    # model.summary()
    return model

In [0]:
def func_obj(params):

    model = get_model(params)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')

    model.fit(
        X_train, 
        y_train,
        batch_size = 128, #int(params.get('batch_size', 128)),
        epochs = 5,
        verbose = 0,
        )
    
    score = model.evaluate(X_test, y_test, verbose=0)
    accuracy = score[1]
    print(params, 'accuracy={}'.format(accuracy))

    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [27]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.3009695203598028, 'dropout_cnn_block_three': 0.4649924345003561, 'dropout_cnn_block_two': 0.3729968123915366, 'dropout_dense_block_one': 0.6458057864184462, 'dropout_dense_block_two': 0.5503427477627053}
accuracy=0.9664399027824402
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.352616534372221, 'dropout_cnn_block_three': 0.31159917446757457, 'dropout_cnn_block_two': 0.468076054790067, 'dropout_dense_block_one': 0.5383150277435664, 'dropout_dense_block_two': 0.4803794434119354}
accuracy=0.9639455676078796
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.41035826538999837, 'dropout_cnn_block_three': 0.4006586387189496, 'dropout_cnn_block_two': 0.4425457623558673, 'dropout_dense_block_one': 0.38691849210534296, 'dropout_dense_block_two': 0.34436409192988104}
accuracy=0.9625850319862366
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3313280756653813, 'dropout_cnn_block_three': 0.4357625984611852, 'dropout_cnn_block_two': 0.42778240028238